In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import IndexSlice as I
import os

from crownetutils.utils.plot import PlotUtil
from crownetutils.omnetpp.scave import CrownetSql
from crownetutils.analysis.omnetpp import OppAnalysis
from crownetutils.analysis.dpmm.builder import DpmmHdfBuilder



In [ ]:

def get_cell_counts(map, hostId):
    """
    Select the density map seen by 'hostId' for all timesteps. Only use the 'selected' 
    values calculated by the used aggregation algorithm and transfrom the map to the 
    following structure:

    columns: [ simtime, cell_1, cell_2, ..., cell_n]
    index: None (simtime is unique and used for x-axis, thus not part of the index)
    """
    host_df = map[I[:, :, :, :, hostId], ["count", "selection"]]
    host_df = host_df[host_df["selection"] != 0].copy(deep=True)
    host_df = host_df.reset_index().drop(columns=["source"])
    host_df = host_df.pivot_table(index=["simtime"], columns=["x", "y"], values="count")
    cell_idx = [f"[{x}, {y}]" for x, y in host_df.columns]
    host_df.columns = host_df.columns.to_flat_index()
    host_df.columns = pd.Index(cell_idx)
    host_df.columns.name = None
    host_df = host_df.reset_index()
    return host_df

def plot_cell_count(ax, host_df):
    for i in range(1, len(host_df.columns)): # all cells without simtime column
        ax.plot('simtime', host_df.columns[i], f"{PlotUtil.plot_color_markers[i]}--", data=host_df, markersize=10)
    ax.set_title(f"Node count in cells in local map of node")
    ax.set_ylabel("Node count")
    ax.set_xlabel("time")
    ax.set_xlim([2.5, 10.0])
    ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.2f'))
    ax.legend()
    return ax

# def get_packet_source_distribution(sql, app_path, normalize=True):
#     """
#     Create square matrix of [hostId X hostId] showing the source hostId of received packets for the given application path.
#     Example:
#     hostId/hostId |  1  |  2  |  3  |
#           1       |  0  |  4  |  8  |
#           2       |  1  |  0  |  1  |
#           3       |  6  |  6  |  0  |
#     host_1 received 4 packets from host_2
#     host_1 received 8 packets from host_3
#     host_2 received 1 packet  from host_1
#     host_2 received 1 packet  from host_3
#     host_3 received 6 packets from host_1
#     host_3 received 6 packets from host_2
#     """
#     id_map = sql.host_ids()
#     df = None
#     for _id, host in id_map.items():
#         _df = sql.vec_merge_on(f"{id_map[_id]}{app_path}", sql.OR(["rcvdPkSeqNo:vector","rcvdPkHostId:vector"]))
#         _df["hostId"] = _id
#         if df is None:
#             df = _df 
#         else:
#             df = pd.concat([df, _df], axis=0)
            

#     df = df.loc[:,["hostId", "rcvdPkHostId:vector"]]
#     df["rcvdPkHostId:vector"] = pd.to_numeric(df["rcvdPkHostId:vector"], downcast="integer")
#     df['val'] = 1
#     df_rcv = df.pivot_table(index="hostId", columns=["rcvdPkHostId:vector"], aggfunc='count', values=["val"], fill_value=0)
#     df_rcv.columns=df_rcv.columns.droplevel()

#     # normalize number of received packets (axis = 1 / over columns)
#     if normalize:
#         _sum = df_rcv.sum(axis=1)
#         _sum = np.repeat(_sum.to_numpy(), _sum.shape[0]).reshape((_sum.shape[0],-1))
#         df_rcv /= _sum
#     return df_rcv

# def plot_packet_source_distribution(ax, data, hatch_patterns=PlotUtil.hatch_patterns):
#     patterns = itertools.cycle(hatch_patterns)

#     ax = data.plot.barh(stacked=True, width=0.5, ax=ax)
#     ax.set_title("Packets received from")
#     ax.set_xlabel("percentage")
#     bars = [i for i in ax.containers if isinstance(i, mpl.container.BarContainer)]
#     for bar in bars:
#         _h = next(patterns)
#         for patch in bar:
#             patch.set_hatch(_h)
#     ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#     return ax


In [ ]:

data_root = f"{os.environ['HOME']}/repos/crownet/crownet/tests/omnetpp/DcDMap_localUpdate_run1/results"
hdf_file = "data.h5"
b = DpmmHdfBuilder.get(hdf_file, data_root)
sql = CrownetSql(vec_path=f"{data_root}/final-#0.vec", 
    sca_path=f"{data_root}/final-#0.sca", 
    network="TestStationaryWorld")
df_cell_count_1 = get_cell_counts(b.build()["map_p"], hostId= 15)
df_source_dist_1 = OppAnalysis.get_packet_source_distribution(sql, app_path=".app[1].app", normalize=True)


data_root = f"{os.environ['HOME']}/repos/crownet/crownet/tests/omnetpp/DcDMap_localUpdate_run2/results"
hdf_file = "data.h5"
b = DpmmHdfBuilder.get(hdf_file, data_root)
sql = CrownetSql(vec_path=f"{data_root}/final-#0.vec", 
    sca_path=f"{data_root}/final-#0.sca", 
    network="TestStationaryWorld")
df_cell_count_2 = get_cell_counts(b.build()["map_p"], hostId= 15)
df_source_dist_2 = OppAnalysis.get_packet_source_distribution(sql, app_path=".app[1].app", normalize=True)

## Time series node count in occupied cells (Single Node View: HostId 15)

In the simulation host `15` stopped sending and receiving beacons at time
$t_{stop} = 5s$. After $t_{TTL} = 3s$ the neighborhood table removed all beacons
still present from other nodes. The count drops at $t=8s$ to zero for all cells
because based on the YMF algorithm the own measurement at time of removing the
old beacons is the newest information. With subsequently received map packets
the density map will use the newer values as the own values get older and will
thus be ignored by the YMF selection.

The map quality is also influenced by synchronized nodes. The graphs in run 2
show vastly different densities because the nodes start at exactly the same time
which leads to unnaturally many collisions. In run 1 the send intervals and start
times are changed based on the given uniform delta $U(min, max)$. Values not 
mentioned in run 2 are identical to run 1.

| Parameter                | Run 1                      | Run2    |
| ------------------------ | -------------------------- | ------- |
| Beacon send interval     | 150 ms + U(0, 10 ms)       | 150 ms  |
| Beacon start time        | U(0, 50 ms)                | 0 ms    |
| Beacon TTL               | 3000 ms                    |         |
| DPD-Map send interval    | 500 ms                     |         |
| DPD-Map start time       | 2001 ms + U(0, 50 ms)      | 2001 ms |
| DPD-Map merge algorithm  | Youngest measurement first |         |
| Node 15: Stop beacon app | 5000ms                     |         |


In [ ]:
fig, axes = plt.subplots(2, 2, figsize= (20, 18))

ax = plot_cell_count(axes[0][0], df_cell_count_1)
ax.set_title(f"Run 1: {ax.get_title()} 15" )

ax = OppAnalysis.plot_packet_source_distribution(axes[1][0], df_source_dist_1)
ax.set_title(f"Run 1: {ax.get_title()}" )

ax = plot_cell_count(axes[0][1], df_cell_count_2)
ax.set_title(f"Run 2: {ax.get_title()} 15" )

ax = OppAnalysis.plot_packet_source_distribution(axes[1][1], df_source_dist_2)
ax.set_title(f"Run 2: {ax.get_title()}" )